In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import max_functions as mf

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

path1 = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=3.0eV' # use your path
path2 = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=4.0eV' # use your path
path3 = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=6.1eV' # use your path
path4 = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=8.2eV' # use your path
path = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=11.5eV' # use your path

all_files = glob.glob(path + "/*.dat")

li = []
cosphi_th = []
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.replace(" ","") # corrects for spaces
    cosphi_th.append((int(df["filename"].str.split("_")[0][0]),int(df["filename"].str.split("_")[0][1])))
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th=np.array(cosphi_th)
frame = pd.concat(li, axis=0)
frame_srt=frame.groupby("filename")

In [ ]:
frame.loc[frame["filename"] == "1_1"] # first way of selecting the right file
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
# Single 2D maps from pandas without tranforming into numpy
frame.loc[frame["filename"] == "1_1"].plot.hexbin(x="phi", y="cos(theta)", C="value", sharex=False, gridsize=50)

In [ ]:
# converting to numpy arrays to plot
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)
counts = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # values

print(phi.shape, ctheta.shape, counts.shape)
print(type(phi), type(ctheta), type(counts))
counts_matrix = counts.reshape(200,100) # reshape(rows=x,columns=y)

print("number of ROWS (x) = ", len(counts_matrix[:,1]), "number of COLUMNS (y for reshaping) = ", len(counts_matrix[1,:]))

In [ ]:
def sorting_array(inarray, cosphi, a):
    """
    a is the level according to which values have to be sorted: 1 = cos_light, 2 = phi_light
    Function to sort the input array according to the labels attached from LMF2ROOT. The lables are stored in cosphi during the loaing.
    To load sorting the 72 items according to cos(theta) use level = 1, according to phi level = 2. The first level is a tring, thereofere
    is not possible to sort according to it.
    NOTE: for experimental data the default order is according to phi, civecersa for the theoretical MFPADs.
    """
    cosn=np.array([col[0] for col in cosphi]); #list
    phin=np.array([col[1] for col in cosphi]); #list
    if inarray.ndim>2:
        data = inarray.reshape(72, inarray.shape[1]*inarray.shape[2]).T
        df = pd.DataFrame(
            data=data,
            #NOTE: phiM and cosM should be passed into the function
            index=pd.MultiIndex.from_product([phiM, cosM],names=["phi","cos(theta)"]),
            columns=['item {}'.format(i) for i in range(72)])

    else:
        cosbin = inarray.shape[1]
        data = inarray.T
        df = pd.DataFrame(
            data=data,
            index=np.linspace(-1,1,inarray.shape[1]),
            columns=['item {}'.format(i) for i in range(72)])

    df1=df.T
    df1["cos_light"]=cosn
    df1["phi_light"]=phin
    df1.set_index("cos_light", append=True, inplace=True) #level=1
    df1.set_index("phi_light", append=True, inplace=True) #level=2

    if inarray.ndim>2:
        dfnew=df1
    else:
        #here it transforms the DataBAse into a series in order to sort correctly the cos(theta) column too
        dfnew=df1.stack()

    #level=2 is the standard for experiment, level=1 for theory
    #kind="mergesort" the concept of stable sorting algorithm
    dfnew.sort_index(level=a, inplace=True, kind="mergesort")
    dfnew.reset_index(level=1, drop=True, inplace=True) # removes cos_light
    dfnew.reset_index(level=1, drop=True, inplace=True) # removes phi

    if inarray.ndim>2:
        #fast approach with numpy with transpose to rearrange the order of the axes:
        outarray=(dfnew.T).values.reshape(inarray.shape[1],inarray.shape[2],72).transpose(2, 0, 1)
    else:
        outarray=dfnew.values.reshape(72, inarray.shape[1])

    return outarray

In [ ]:
import matplotlib as mpl
from matplotlib import cm

#the following converts a cmap into plotly language 
magma_cmap = mpl.cm.get_cmap('magma')
seismic_cmap = mpl.cm.get_cmap('seismic')

magma_rgb = []
seismic_rgb = []
norm = mpl.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(magma_cmap(norm(i)))
       magma_rgb.append(k)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
       seismic_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

Magma_r = matplotlib_to_plotly(magma_cmap, 255)
Seismic_r = matplotlib_to_plotly(seismic_cmap, 255)

In [ ]:
# plt.imshow(z_matrix)
plt.imshow(counts_matrix.T,extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], aspect=180)
ax = plt.axes()
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
plt.colorbar()

In [ ]:
# countour plot
cmap = mpl.cm.viridis
cmap.set_under('w')


fig1, ax = plt.subplots(constrained_layout=True)

CS = ax.contourf(phi,ctheta,counts_matrix.T, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
fig1.colorbar(CS)

# OLD WAY OF PROCEDING!! Note unique values and extent, all the 72 plots
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

fig = plt.figure(figsize=(50, 24),dpi=80) # figsize in inche by defauls: 1 in = 2.8 cm
for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax = fig.add_subplot(6, 12, i)
    ax.contourf(phi,ctheta,counts_matrix, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
    ax.text(i, 1+i//12, i, ha="center", va="center", color="red")

fig.tight_layout()
# plt.show()

#OLD WAY OF PROCEEDING

fig = plt.figure(figsize=(25, 12),dpi=40) # downsized
for filename,group,i in zip(frame_srt,frame_srt,range(72)):
    counts_str = group[1]["value"].to_numpy()
    ax = fig.add_subplot(6, 12, i+1)
    ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    ax.text(i+1, i//12+1, cosphi_th[i], ha="center", va="center", color="red")

fig.tight_layout()

In [ ]:
cosphi_th;

In [ ]:
# all the 72 plots
# the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
# 6 levels of cos(theta) (y), 12 levels of phi (x)

#NOTE: the following phi and ctheta are not unique: len=20000
phi_full = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta_full = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

for filename,group,i,ax in zip(frame_srt,frame_srt,range(72),axes.flatten()) :
    counts_str = group[1]["value"].to_numpy()
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    ax.text(0, 0, cosphi_th[i], ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()


# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :

# OLD WAY OF PROCEDING!! Note unique values and extent, all the 72 plots
    phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
    ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

    fig = plt.figure(figsize=(50, 24),dpi=80) # figsize in inche by defauls: 1 in = 2.8 cm
    for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
        counts_str = group[1]["value"].to_numpy()
        counts_matrix = counts_str.reshape(200,100)
        ax = fig.add_subplot(6, 12, i)
        ax.contourf(phi,ctheta,counts_matrix, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
        ax.text(i, 1+i//12, i, ha="center", va="center", color="red")
    
    fig.tight_layout()
    # plt.show()"

  #OLD WAY OF PROCEEDING
    fig = plt.figure(figsize=(25, 12),dpi=40) # downsized
    for filename,group,i in zip(frame_srt,frame_srt,range(72)):
        counts_str = group[1]["value"].to_numpy()
        ax = fig.add_subplot(6, 12, i+1)
        ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
        ax.text(i+1, i//12+1, cosphi_th[i], ha="center", va="center", color="red")
    fig.tight_layout()

In [ ]:
# all the 72 plots: no borders & sharing axis

phi_full = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta_full = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)

# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row') figsize in inche by defauls: 1 in = 2.8 cm
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row') #downsized
fig.subplots_adjust(hspace = .5, wspace=.5)

for filename,group,ax in zip(frame_srt,frame_srt,axes.flatten()) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    
fig.tight_layout()
plt.show()

In [ ]:
#standard sortin according to cos
from numpy.core.defchararray import multiply

MFPAD_S_cos=[]
MFPAD_R_cos=[]

cos_projthS_cos=[]
cos_projthR_cos=[]
phi_projthS_cos=[]
phi_projthR_cos=[]

for filename,group,i in zip(frame_srt,frame_srt,range(72)):
    MFPAD_S_cos.append(group[1]["value"].to_numpy().reshape(200,100))
    cos_projthS_cos.append(MFPAD_S_cos[i].sum(axis=0))
    phi_projthS_cos.append(MFPAD_S_cos[i].sum(axis=1))
    tempcos=np.flip(MFPAD_S_cos[i], axis=0)
    tempphi=np.flip(tempcos, axis=1)
    MFPAD_R_cos.append(tempphi)
    cos_projthR_cos.append(MFPAD_R_cos[i].sum(axis=0))
    phi_projthR_cos.append(MFPAD_R_cos[i].sum(axis=1))

cos_projthS_cos=np.array(cos_projthS_cos)
cos_projthR_cos=np.array(cos_projthR_cos)
phi_projthS_cos=np.array(phi_projthS_cos)
phi_projthR_cos=np.array(phi_projthR_cos)

print(np.array(cos_projthS_cos).shape)
print(np.array(cos_projthR_cos).shape)

In [ ]:
MFPAD_S_cos_norm=[]
MFPAD_R_cos_norm=[]
mPECD_cos=[]

for cols,colr in zip(MFPAD_S_cos,MFPAD_R_cos):
    MFPAD_S_cos_norm.append(cols/cols.sum(axis=1)[:,np.newaxis])
    MFPAD_R_cos_norm.append(colr/colr.sum(axis=1)[:,np.newaxis])

MFPAD_S_cos_norm=np.array(MFPAD_S_cos_norm)
MFPAD_R_cos_norm=np.array(MFPAD_R_cos_norm)
MFPAD_S_cos_norm_sum=np.sum(MFPAD_S_cos_norm,axis=0)
MFPAD_R_cos_norm_sum=np.sum(MFPAD_R_cos_norm,axis=0)

mPECD_cos=np.divide(np.subtract(MFPAD_R_cos_norm,MFPAD_S_cos_norm),np.add(MFPAD_R_cos_norm,MFPAD_S_cos_norm))
mPECD_cos_sum=np.divide(np.subtract(MFPAD_R_cos_norm_sum,MFPAD_S_cos_norm_sum),np.add(MFPAD_R_cos_norm_sum,MFPAD_S_cos_norm_sum))

In [ ]:
cPECD_cos =  np.divide(np.subtract(cos_projthR_cos,cos_projthS_cos),np.add(cos_projthR_cos,cos_projthS_cos))
#it is somehow unconventional, but let's normalise the error with the square root of all counts
# errPECD = np.sqrt(1/(cos_projthR+cos_projthS))/np.sqrt(np.sum(cos_projthR))
errPECD = np.sqrt(1/(cos_projthR_cos+cos_projthS_cos))

In [ ]:
print(np.sum(cos_projthR_cos[60]))
plt.plot(errPECD[60])
plt.plot(cPECD_cos[60])
plt.plot(cos_projthR_cos[60])
plt.plot(cos_projthS_cos[60])

In [ ]:
plt.plot(cos_projthR_cos[0])
plt.plot(cos_projthS_cos[0])

In [ ]:
# modified with error
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2/(self.err**2))
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

def PECD6(x, b1, b2, b3, b4, b5, b6):
    return ((b1*x + 0.5*b3*(5*x**3-3*x) +0.125*b5*(63*x**5-70*x**3+15*x)) / (1 + 0.5*b2*(3*x**2-1) + 0.125*b4*(35*x**4-30**2+3) + 0.0625*b6*(231*x**6-315*x**4+105*x**2-5)))

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline

param_matrix_cos = [[0 for i in range(72)] for j in range(6)] #i col, j row

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2)

for i in range(72): #72 fixed because of the photon directions, equal to range(len(cosphi))
    x_data = np.flip(ctheta)
    y_data = cPECD_cos[i]
    y_err = errPECD[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
    m.migrad() # run optimiser
    m.hesse() # run covariance estimator

    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
    #NOTE: x_data has to be monotonicalli INCREASING!
    bspl = make_interp_spline(x_data, PECD6(x_data, *m.values.values()), k=5)
    
    ax = fig.add_subplot(6, 12, i+1)
    ax.plot(x_data_new,bspl(x_data_new))
    ax.scatter(x_data,y_data,color='r')
    ax.set_xlim(custom_xlim)
    ax.set_ylim(custom_ylim)
    # ax.errorbar(x_data,y_data, yerr=y_err, fmt="o")

    for j,p in zip(range(len(m.parameters)),m.parameters): # len = 6
        param_matrix_cos[j][i] = ((m.values[p], m.errors[p]))

fig.tight_layout()
plt.show()
param_matrix_cos=np.array(param_matrix_cos) #numpy tranformation!!!    

In [ ]:
print(param_matrix_cos.shape)
param_matrix_cos[0,:,0].shape

In [ ]:
from numpy.lib.function_base import meshgrid
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

param_matrix_cos=np.array(param_matrix_cos) #numpy tranformation!!!
phicos_MFPAD = np.array(list(itertools.product(np.linspace(-180,180,12).tolist(), np.linspace(-1,1,6).tolist())))

go_phi=[col[0] for col in phicos_MFPAD]
go_cos=[col[1] for col in phicos_MFPAD]

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r), 1, 1)
fig.add_trace(go.Contour(z=param_matrix_cos[0,:,0], x=go_phi, y=go_cos, line_smoothing=0.75, colorscale=Seismic_r), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
# it is already the middle point, no need of doing it again
phiM = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
cosM = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

# phiM=(phi[1:] + phi[:-1])/2
# cosM=(ctheta[1:] + ctheta[:-1])/2

print(phiM.shape,cosM.shape)

In [ ]:
# convert the counts into a 2D np array and sort according to cos and phi

counts_temp=[] #standard cos sorted
counts_matrixS_phi=[]
counts_matrixS_cos=[]
counts_matrixR_phi=[]
counts_matrixR_cos=[]

#respect to cs is the standard calculation
cos_projthS_phi=[]
phi_projthS_phi=[]

cos_projthR_phi=[]
phi_projthR_phi=[]

for filename,group,i in zip(frame_srt,frame_srt,range(72)):
    counts_temp.append(group[1]["value"].to_numpy().reshape(200,100))
    
counts_temp=np.array(counts_temp)
counts_matrixS_phi=sorting_array(counts_temp, cosphi_th, 2) #sorted according to phi
counts_matrixS_cos=sorting_array(counts_temp, cosphi_th, 1) #sorted according to cos

for j,el in enumerate(counts_temp):
    #projection for S enantiomer 
    cos_projthS_phi.append(counts_matrixS_phi[j].sum(axis=0))
    phi_projthS_phi.append(counts_matrixS_phi[j].sum(axis=1))

    #flipping for R enantiomer
    tempcos_phi=np.flip(counts_matrixS_phi[j], axis=0)
    tempcos_cos=np.flip(counts_matrixS_cos[j], axis=0)
    counts_matrixR_phi.append(np.flip(tempcos_phi, axis=1))
    counts_matrixR_cos.append(np.flip(tempcos_phi, axis=1))
    #projection for R enantiomer 
    cos_projthR_phi.append(counts_matrixR_phi[j].sum(axis=0))
    phi_projthR_phi.append(counts_matrixR_phi[j].sum(axis=1))


#projections
cos_projthS_phi=np.array(cos_projthS_phi)
phi_projthS_phi=np.array(phi_projthS_phi)
cos_projthR_phi=np.array(cos_projthR_phi)
phi_projthR_phi=np.array(phi_projthR_phi)

counts_matrixR_phi=np.array(counts_matrixR_phi)
counts_matrixR_cos=np.array(counts_matrixR_cos)

print(np.array(counts_matrixS_phi).shape)
print(np.array(counts_matrixR_phi).shape)
print(np.array(counts_matrixR_cos).shape)
print(np.array(cos_projthS_phi).shape)
print(np.array(cos_projthR_phi).shape)

In [ ]:
# all the 72 plots FOR R ENANTIOMER NOT SORTED: to be compare directly with experimental data
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

for filename,group,i,ax in zip(frame_srt,frame_srt,range(72),axes.flatten()) :
    counts_str = group[1]["value"].to_numpy()
    ax.contourf(phi_full.reshape(200,100),ctheta_full.reshape(200,100),counts_matrixR_cos[i].reshape(200,100), levels=20, vmin=0.0001)
    ax.text(0, 0, cosphi_th[i], ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

In [ ]:
cPECD_phi =  np.divide(np.subtract(cos_projthR_phi,cos_projthS_phi),np.add(cos_projthR_phi,cos_projthS_phi))
errPECD_phi = np.sqrt(1/(cos_projthR_phi+cos_projthS_phi))

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline

param_matrix_phi = [[0 for i in range(72)] for j in range(6)] #i col, j row

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2)

for i in range(72): #fixe because of the photon directions, equal to range(len(cosphi))
    #print(i)
    x_data = np.flip(ctheta)
    y_data = cPECD_phi[i]
    y_err = errPECD_phi[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
    m.migrad() # run optimiser
    m.hesse() # run covariance estimator

    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
    bspl = make_interp_spline(x_data, PECD6(x_data, *m.values.values()), k=5)
    
    ax = fig.add_subplot(6, 12, i+1)
    ax.plot(x_data_new, bspl(x_data_new))
    ax.scatter(x_data,y_data,color='r')
    ax.set_xlim(custom_xlim)
    ax.set_ylim(custom_ylim)
    # ax.errorbar(x_data,y_data, yerr=y_err, fmt="o")

    for j,p in zip(range(len(m.parameters)),m.parameters): # len = 6
        param_matrix_phi[j][i] = ((m.values[p], m.errors[p]))

fig.tight_layout()
plt.show()
param_matrix_phi=np.array(param_matrix_phi) #numpy tranformation!!!    

In [ ]:
#to be compared with the b1 maps of the experimental data AS THEY ARE (not sorted)

fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(len(param_matrix_phi)):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix_phi[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_phi[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=Seismic_r, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix_phi[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, legendgroup="yes", contours=dict(start=-0.2, end=0.2, size=0.04)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix_phi[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=Seismic_r, legendgroup="yes", contours=dict(start=-0.2, end=0.2, size=0.04)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
#INTEGRATED S AND R MFPADs
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,figsize=(12, 6))

ax0.imshow(MFPAD_R_cos_norm_sum, extent=[go_phi[0],go_phi[-1],go_cos[0],go_cos[-1]], interpolation='quadric', cmap='seismic', aspect="auto")
ax0.set_title("MFPAD R enant")

ax1.imshow(MFPAD_S_cos_norm_sum, extent=[go_phi[0],go_phi[-1],go_cos[0],go_cos[-1]], interpolation='quadric', cmap='seismic', aspect="auto")
ax1.set_title("MFPAD S enant")

plt.show()

In [ ]:
#extent= [go_phi[0],go_phi[-1],go_cos[0],go_cos[-1]]

# interpolation: [ 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'.]

fig=plt.figure()
plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, extent=[go_phi[0],go_phi[-1],go_cos[0],go_cos[-1]], interpolation='none', cmap='seismic', aspect="auto")
plt.title("Contrast map theory")
plt.colorbar()

fig1=plt.figure()
plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, extent=[go_phi[0],go_phi[-1],go_cos[0],go_cos[-1]], interpolation='bilinear', cmap='seismic', aspect="auto")
plt.title("Contrast map theory bilinear")
plt.colorbar()

fig2=plt.figure()
plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, extent=[go_phi[0],go_phi[-1],go_cos[0],go_cos[-1]], interpolation='quadric', cmap='seismic', aspect="auto")
plt.title("Contrast map theory quadratic interpolation")
plt.colorbar()